In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
root_path = os.path.abspath('./..')
sys.path.insert(0, root_path )

import itertools
import functools
from tqdm import tqdm

import pandas as pd
import numpy as np
import numba
import sklearn
import sklearn.linear_model
import sklearn.cluster
import sklearn.metrics
import matplotlib.pyplot as plt

import hiddensc
from hiddensc import utils, files, vis

import scanpy as sc
import scvi
import anndata

utils.set_random_seed(utils.RANDOM_SEED)
utils.print_module_versions([sc, anndata, scvi, hiddensc])
vis.visual_settings()

Random seed set to 42
scanpy              : 1.9.3
anndata             : 0.8.0
scvi                : 0.20.3
hiddensc            : beta_25.03.23


# Load data

In [6]:
OVERWRITE=False
fname = os.path.join(root_path, 'figures', 'ablation', 'optimal_NUM_PCS_KS_dict.npz')
optimal_ks = files.load_npz(fname)

exp_id = 1404
utils.set_random_seed(utils.RANDOM_SEED)
data_name = f'naiveB_1900_memoryB_{exp_id:d}'
# Utility filename makers.
at_results_dir = functools.partial(os.path.join, root_path, files.RESULT_DIR, data_name)
at_train_dir = functools.partial(os.path.join, root_path, files.RESULT_DIR, data_name, 'training')
os.makedirs(at_results_dir(), exist_ok=True)
os.makedirs(at_train_dir(), exist_ok=True)
fname = at_results_dir('dim_reduced.npz')
if not OVERWRITE and os.path.exists(fname):
    print(f'Skipping {data_name}, found outputs and overwrite={OVERWRITE}')

# Load dataset.
main_datafile = os.path.join(root_path, files.DATA_DIR, f'{data_name}_raw.h5ad')
adata = sc.read(main_datafile)
hiddensc.datasets.preprocess_data(adata)

Random seed set to 42


In [21]:
# Features.
feats = {}
x_pca = hiddensc.models.get_pca(adata)
k = optimal_ks[data_name]
feats['PCA'] = x_pca[:, :k]

In [24]:
ks = [25]
n_epochs = 250

model_classes = [scvi.model.LinearSCVI, scvi.model.SCVI]
combos = list(itertools.product(model_classes, ks))
for model_cls, k in tqdm(combos):
    local_adata = adata.copy()
    name = f'{model_cls.__name__}_{k}'
    model_cls.setup_anndata(local_adata, layer="counts")
    model = model_cls(local_adata, n_latent=k)
    model.train(max_epochs=n_epochs, plan_kwargs={"lr": 5e-3},
                check_val_every_n_epoch=5)
    train_elbo = model.history["elbo_train"][1:]
    test_elbo = model.history["elbo_validation"]
    ax = train_elbo.plot()
    test_elbo.plot(ax=ax)
    plt.yscale('log')
    plt.savefig(at_train_dir(f'{name}.png'))
    plt.title(name)
    feats[name] = model.get_latent_representation()
    plt.clf()  # plt.show()
    del local_adata

  0%|          | 0/2 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 250/250: 100%|██████████| 250/250 [01:22<00:00,  2.97it/s, loss=2.36e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=250` reached.


Epoch 250/250: 100%|██████████| 250/250 [01:22<00:00,  3.03it/s, loss=2.36e+03, v_num=1]


 50%|█████     | 1/2 [01:24<01:24, 84.05s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

`Trainer.fit` stopped: `max_epochs=250` reached.


Epoch 250/250: 100%|██████████| 250/250 [01:25<00:00,  2.91it/s, loss=2.08e+03, v_num=1]


100%|██████████| 2/2 [02:51<00:00, 85.96s/it]


<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

In [23]:
feats

{'PCA': array([[ -1.1196005 ,   6.622503  ,  -2.5259159 , ...,  -1.833579  ,
          -2.2876647 ,  -0.6665094 ],
        [ -7.131651  ,   1.0248331 ,   1.5389104 , ...,  -0.9276155 ,
          -1.5275725 ,  -2.2917721 ],
        [-10.160202  ,   0.3727494 ,   6.830382  , ...,   1.0274894 ,
          -2.3209364 ,   0.53815734],
        ...,
        [  5.3826294 ,  -1.4552354 ,  -0.98211735, ...,   1.6789066 ,
           3.2856157 ,   4.929672  ],
        [  8.764949  ,  -5.606997  ,  -1.1607517 , ...,  -5.339203  ,
           0.03333269,   0.6223444 ],
        [  6.564807  ,  -3.4688787 ,  -3.5198483 , ...,  -0.6038395 ,
          -2.4636164 ,   0.09449834]], dtype=float32),
 'LinearSCVI_10': array([[-0.02679488, -0.1052369 , -1.7476829 , ..., -1.3801131 ,
         -1.6175925 ,  1.0072393 ],
        [-1.1210295 ,  0.0494108 , -1.507539  , ...,  0.70321447,
         -0.13254239,  0.81325614],
        [ 1.3983492 , -0.67633283, -2.347671  , ..., -1.1255838 ,
         -0.37167725,  0.246

In [25]:
feats

{'PCA': array([[ -1.1196005 ,   6.622503  ,  -2.5259159 , ...,  -1.833579  ,
          -2.2876647 ,  -0.6665094 ],
        [ -7.131651  ,   1.0248331 ,   1.5389104 , ...,  -0.9276155 ,
          -1.5275725 ,  -2.2917721 ],
        [-10.160202  ,   0.3727494 ,   6.830382  , ...,   1.0274894 ,
          -2.3209364 ,   0.53815734],
        ...,
        [  5.3826294 ,  -1.4552354 ,  -0.98211735, ...,   1.6789066 ,
           3.2856157 ,   4.929672  ],
        [  8.764949  ,  -5.606997  ,  -1.1607517 , ...,  -5.339203  ,
           0.03333269,   0.6223444 ],
        [  6.564807  ,  -3.4688787 ,  -3.5198483 , ...,  -0.6038395 ,
          -2.4636164 ,   0.09449834]], dtype=float32),
 'LinearSCVI_10': array([[-0.02679488, -0.1052369 , -1.7476829 , ..., -1.3801131 ,
         -1.6175925 ,  1.0072393 ],
        [-1.1210295 ,  0.0494108 , -1.507539  , ...,  0.70321447,
         -0.13254239,  0.81325614],
        [ 1.3983492 , -0.67633283, -2.347671  , ..., -1.1255838 ,
         -0.37167725,  0.246

Prepare directories

In [26]:
np.savez_compressed(fname, **feats)

In [7]:

fname = os.path.join( root_path, 'figures', 'ablation', 'optimal_NUM_PCS_KS_dict.npz')
optimal_ks = files.load_npz(fname)
k = optimal_ks[exp_name]


array(49)

# Generate dim reduced features

In [5]:
feats = {}

## PCA

In [6]:
x_pca = hiddensc.models.get_pca(adata)
feats['PCA'] = x_pca

## SCVI / LinearSCVI

In [7]:
n_epochs = 250

model_classes = [scvi.model.LinearSCVI, scvi.model.SCVI]
ks = [10]#, 20, 30, 40, 50]
combos = list(itertools.product(model_classes, ks))

for model_cls, k in tqdm(combos):
    local_adata = adata.copy()
    name = f'{model_cls.__name__}_{k}'
    model_cls.setup_anndata(local_adata, layer="counts")
    model = model_cls(local_adata, n_latent=k)
    model.train(max_epochs=n_epochs, plan_kwargs={"lr": 5e-3}, check_val_every_n_epoch=5)
    train_elbo = model.history["elbo_train"][1:]
    test_elbo = model.history["elbo_validation"]
    ax = train_elbo.plot()
    test_elbo.plot(ax=ax)
    plt.yscale('log')
    plt.savefig(at_train_dir(f'{name}.png'))
    plt.title(name)
    feats[name] = model.get_latent_representation()
    #plt.show()
    plt.clf()
    del local_adata

  0%|                                                     | 0/2 [00:00<?, ?it/s]No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 250/250: 100%|██| 250/250 [17:32<00:00,  4.22s/it, loss=2.23e+03, v_num=1]

`Trainer.fit` stopped: `max_epochs=250` reached.


Epoch 250/250: 100%|██| 250/250 [17:32<00:00,  4.21s/it, loss=2.23e+03, v_num=1]


 50%|█████████████████████▌                     | 1/2 [17:35<17:35, 1055.01s/it]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Epoch 216/250:  86%|▊| 215/250 [2:08:54<2:18:30, 237.45s/it, loss=1.89e+03, v_nu

/home/beangoben/miniconda/envs/hidden/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
100%|█████████████████████████████████████████| 2/2 [2:26:32<00:00, 4396.33s/it]


<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>

In [11]:
fname = at_results_dir('features.npz')
np.savez_compressed(fname, **feats)

# Generate predictions

In [9]:
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.simplefilter("ignore", category=ConvergenceWarning)


feats = files.load_npz(at_results_dir('features.npz'))
y = (adata.obs['batch'].values == 'Case').astype(np.int32)
y_true = (adata.obs['perturbed'].values == 'Memory B').astype(np.int32)
ids = adata.obs['barcodes'].values
pred_fns = {'linear': hiddensc.models.linear_predictions,
              'svm': hiddensc.models.svm_predictions}

preds = [y, y_true]
info = [('batch', '','Case'), ('perturbed', '','Memory B')]
combos = list(itertools.product(feats.keys(), pred_fns.keys()))

for feat_name, strat_name  in tqdm(combos):
    rand_state=0
    x = feats[feat_name]
    p_hat, p_labels = pred_fns[strat_name](x, y, 1, rand_state)
    preds.append(p_hat)
    info.append((feat_name, strat_name, 'p_hat'))
    preds.append(p_labels)
    info.append((feat_name, strat_name, 'p_label'))
    
cols = pd.MultiIndex.from_tuples(info)
pred_df = pd.DataFrame(np.array(preds).T, index=adata.obs['barcodes'], columns=cols)
pred_df.to_csv(at_results_dir('predictions.csv'))
pred_df


100%|█████████████████████████████████████████████| 6/6 [00:29<00:00,  5.00s/it]


batch perturbed       PCA                    \
                                                linear               svm   
                              Case  Memory B     p_hat p_label     p_hat   
barcodes                                                                   
naiveB_a_AAACCTGCACGGTAGA-1    1.0       0.0  0.493125     0.0  0.500000   
naiveB_a_AAACCTGCAGATGGGT-1    1.0       0.0  0.395382     0.0  0.468766   
naiveB_a_AAAGATGCATTTCAGG-1    0.0       0.0  0.359442     0.0  0.464822   
naiveB_a_AAAGCAAAGCCAACAG-1    0.0       0.0  0.478047     0.0  0.500000   
naiveB_a_AAAGCAAAGTGCCATT-1    1.0       0.0  0.512292     0.0  0.500000   
...                            ...       ...       ...     ...       ...   
memoryB_b_TCAGCTCAGCGTGTCC-1   1.0       1.0  0.674898     1.0  0.534335   
memoryB_b_TCATTTGAGTCACGCC-1   1.0       1.0  0.676409     1.0  0.532120   
memoryB_b_TCATTTGTCAGTTGAC-1   1.0       1.0  0.648580     1.0  0.532733   
memoryB_b_TCGAGGCGTTACGGAG-1   1.0       1.0  0.701302     1.0  0.544463   
memoryB_b_TGGCGCAAGCAGATCG-1   1.0       1.0  0.560128     1.0  0.513558   

                                     LinearSCVI_10                            \
                                            linear               svm           
                             p_label         p_hat p_label     p_hat p_label   
barcodes                                                                       
naiveB_a_AAACCTGCACGGTAGA-1      1.0      0.522950     1.0  0.519707     1.0   
naiveB_a_AAACCTGCAGATGGGT-1      0.0      0.432583     0.0  0.470591     0.0   
naiveB_a_AAAGATGCATTTCAGG-1      0.0      0.461520     0.0  0.487306     0.0   
naiveB_a_AAAGCAAAGCCAACAG-1      0.0      0.443951     0.0  0.470765     0.0   
naiveB_a_AAAGCAAAGTGCCATT-1      1.0      0.516501     0.0  0.500000     0.0   
...                              ...           ...     ...       ...     ...   
memoryB_b_TCAGCTCAGCGTGTCC-1     1.0      0.655323     1.0  0.549441     1.0   
memoryB_b_TCATTTGAGTCACGCC-1     1.0      0.520891     1.0  0.493158     0.0   
memoryB_b_TCATTTGTCAGTTGAC-1     1.0      0.561100     1.0  0.524815     1.0   
memoryB_b_TCGAGGCGTTACGGAG-1     1.0      0.724840     1.0  0.587956     1.0   
memoryB_b_TGGCGCAAGCAGATCG-1     1.0      0.562956     1.0  0.524924     1.0   

                               SCVI_10                            
                                linear               svm          
                                 p_hat p_label     p_hat p_label  
barcodes                                                          
naiveB_a_AAACCTGCACGGTAGA-1   0.525948     1.0  0.511121     1.0  
naiveB_a_AAACCTGCAGATGGGT-1   0.430635     0.0  0.469353     0.0  
naiveB_a_AAAGATGCATTTCAGG-1   0.540256     0.0  0.511193     1.0  
naiveB_a_AAAGCAAAGCCAACAG-1   0.463140     0.0  0.489425     0.0  
naiveB_a_AAAGCAAAGTGCCATT-1   0.408716     0.0  0.459262     0.0  
...                                ...     ...       ...     ...  
memoryB_b_TCAGCTCAGCGTGTCC-1  0.552394     1.0  0.515062     1.0  
memoryB_b_TCATTTGAGTCACGCC-1  0.584988     1.0  0.528655     1.0  
memoryB_b_TCATTTGTCAGTTGAC-1  0.567481     1.0  0.527113     1.0  
memoryB_b_TCGAGGCGTTACGGAG-1  0.598753     1.0  0.534616     1.0  
memoryB_b_TGGCGCAAGCAGATCG-1  0.497072     0.0  0.500000     0.0  

[1949 rows x 14 columns]

# Generate performance stats

In [12]:
batch, perturbed, pred_df = files.load_predictions(at_results_dir('predictions.csv'))
results = []
for (dim_reduce, method), grp_df in pred_df.groupby(axis=1, level=[0,1]):
    p_hat = grp_df[(dim_reduce, method, 'p_hat')]
    p_label = grp_df[(dim_reduce, method, 'p_label')]
    stat = {'exp':exp_name, 
            'dim_reduce': dim_reduce,
            'method': method}
    stat.update(hiddensc.metrics.evaluate_classification(perturbed, p_hat, p_label))
    results.append(stat)

results_df = pd.DataFrame(results)
results_df.to_csv(at_results_dir('classification_stats.csv'), index=False)
results_df 

,exp,dim_reduce,method,AUCROC,AP,Recall,Precision,F1
0,naiveB_1900_memoryB_49,LinearSCVI_10,linear,0.920827,0.339665,0.938776,0.130682,0.229426
1,naiveB_1900_memoryB_49,LinearSCVI_10,svm,0.888733,0.288445,0.877551,0.059474,0.111399
2,naiveB_1900_memoryB_49,PCA,linear,0.896885,0.211313,0.918367,0.104651,0.187891
3,naiveB_1900_memoryB_49,PCA,svm,0.863663,0.143348,0.938776,0.049516,0.094070
4,naiveB_1900_memoryB_49,SCVI_10,linear,0.807981,0.143701,0.795918,0.098485,0.175281
5,naiveB_1900_memoryB_49,SCVI_10,svm,0.769882,0.113433,0.816327,0.043956,0.083420


In [10]:
dict(np.load(at_figure_dir('DE_results.npz'), allow_pickle=True))

{'naive_DEs': array(['IGHD', 'TCL1A', 'CXCR4', 'CD74', 'BTG1', 'RPL18A'], dtype=object),
 'memory_DEs': array(['B2M', 'COTL1', 'RPS14', 'EEF1A1', 'ITGB1', 'IGHA1', 'CLECL1'],
       dtype=object),
 'control_DEs': array([], dtype=object),
 'case_DEs': array([], dtype=object),
 'HiDDEN_0_DEs': array(['BTG1', 'CXCR4', 'TMSB4X', 'MT-ATP6', 'IGHD', 'HMGB1'],
       dtype=object),
 'HiDDEN_1_DEs': array(['EEF2', 'EEF1A1', 'RPS4X', 'SLC25A6', 'RPL23A'], dtype=object)}

# Get DE gene statistics

First "true" genes

In [13]:
de_genes = hiddensc.datasets.get_de_genes(adata, 'perturbed')
de_genes.update(hiddensc.datasets.get_de_genes(adata, 'batch'))
de_genes

{'Memory B': array(['B2M', 'COTL1', 'RPS14', 'EEF1A1', 'ITGB1', 'IGHA1', 'CLECL1',
        'HLA-C'], dtype='<U6'),
 'Naive B': array(['IGHD', 'TCL1A', 'CXCR4', 'CD74', 'BTG1', 'RPL18A'], dtype='<U6'),
 'Case': array([], dtype='<U1'),
 'Control': array([], dtype='<U1')}

In [14]:
results = []
info = {'exp_name':exp_name,
        'True labels': 'Naive B/Memory B',
        'Compare_labels': 'Control/Case', 'info':''}
info.update(hiddensc.metrics.evaluate_de_genes(de_genes, 'Naive B', 'Memory B', 'Control', 'Case'))
results.append(info)

In [15]:
NEW_LABEL = 'HiDDEN'
best_model = ('PCA', 'linear', 'p_label')

adata.obs['refined_labels'] = pred_df[best_model].values.astype(int).astype(str)
de_genes.update(hiddensc.datasets.get_de_genes(adata, 'refined_labels', f'{NEW_LABEL}_'))

info = {'exp_name':exp_name,
        'True labels': 'Naive B/Memory B',
        'Compare_labels': NEW_LABEL, 'info': '_'.join(best_model[:2])}
info.update(hiddensc.metrics.evaluate_de_genes(de_genes, 'Naive B', 'Memory B', f'{NEW_LABEL}_0', f'{NEW_LABEL}_1' ))
info.update({})
results.append(info)
de_genes

{'Memory B': array(['B2M', 'COTL1', 'RPS14', 'EEF1A1', 'ITGB1', 'IGHA1', 'CLECL1',
        'HLA-C'], dtype='<U6'),
 'Naive B': array(['IGHD', 'TCL1A', 'CXCR4', 'CD74', 'BTG1', 'RPL18A'], dtype='<U6'),
 'Case': array([], dtype='<U1'),
 'Control': array([], dtype='<U1'),
 'HiDDEN_0': array(['BTG1', 'MT-ATP6', 'CD74', 'TMSB4X', 'MT-CO2', 'CXCR4', 'IGHD'],
       dtype='<U7'),
 'HiDDEN_1': array(['EEF2', 'EEF1A1', 'RPS4X', 'SLC25A6', 'RPL23A', 'RPS8', 'RPL11'],
       dtype='<U7')}

In [16]:
NEW_LABEL = 'LSCVI'
best_model = ('LinearSCVI_10', 'linear', 'p_label')

adata.obs['refined_labels'] = pred_df[best_model].values.astype(int).astype(str)
de_genes.update(hiddensc.datasets.get_de_genes(adata, 'refined_labels', f'{NEW_LABEL}_'))

info = {'exp_name':exp_name,
        'True labels': 'Naive B/Memory B',
        'Compare_labels': NEW_LABEL, 'info': '_'.join(best_model[:2])}
info.update(hiddensc.metrics.evaluate_de_genes(de_genes, 'Naive B', 'Memory B', f'{NEW_LABEL}_0', f'{NEW_LABEL}_1' ))
info.update({})
results.append(info)
de_genes

{'Memory B': array(['B2M', 'COTL1', 'RPS14', 'EEF1A1', 'ITGB1', 'IGHA1', 'CLECL1',
        'HLA-C'], dtype='<U6'),
 'Naive B': array(['IGHD', 'TCL1A', 'CXCR4', 'CD74', 'BTG1', 'RPL18A'], dtype='<U6'),
 'Case': array([], dtype='<U1'),
 'Control': array([], dtype='<U1'),
 'HiDDEN_0': array(['BTG1', 'MT-ATP6', 'CD74', 'TMSB4X', 'MT-CO2', 'CXCR4', 'IGHD'],
       dtype='<U7'),
 'HiDDEN_1': array(['EEF2', 'EEF1A1', 'RPS4X', 'SLC25A6', 'RPL23A', 'RPS8', 'RPL11'],
       dtype='<U7'),
 'LSCVI_0': array(['IGHD', 'CXCR4', 'CD74', 'BTG1', 'TCL1A', 'IGLC2', 'MEF2C',
        'TMSB4X', 'SEC62', 'IGLC3', 'HLA-DRB1'], dtype='<U8'),
 'LSCVI_1': array(['EEF1A1', 'EEF2', 'B2M', 'RPL23A', 'RPS14', 'RPS18', 'RPL27A',
        'COTL1', 'PLAC8', 'RPS8', 'RPL19', 'RPS15A', 'HLA-C', 'RPS23',
        'JUNB', 'TPT1', 'GNB2L1', 'LY6E', 'DUSP1', 'RPS4X'], dtype='<U6')}

In [17]:
de_df = pd.DataFrame(results)
de_df

,exp_name,True labels,Compare_labels,info,precision,recall,fdr,f1_score
0,naiveB_1900_memoryB_49,Naive B/Memory B,Control/Case,,NaN,0.000000,NaN,NaN
1,naiveB_1900_memoryB_49,Naive B/Memory B,HiDDEN,PCA_linear,0.357143,0.357143,0.642857,0.357143
2,naiveB_1900_memoryB_49,Naive B/Memory B,LSCVI,LinearSCVI_10_linear,0.322581,0.714286,0.677419,0.444444


In [20]:
files.save_npz(at_results_dir('de_genes.npz'), de_genes)
de_df.to_csv(at_results_dir('de_genes.csv'))